In [289]:
import discord
import random
import os
import asyncio

import pandas as pd
import requests

#########################

import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import random
from collections import Counter, defaultdict

## function


In [290]:
# Helper function to determine the phase of an over
def get_phase(over):
    if over <= 6:
        return 'P'
    elif over <= 15:
        return 'M'
    else:
        return 'D'

# Function to generate the bowling sequence
def generate_bowling_order(bowler_dict, max_overs=4):
    # Phase-wise bowler allocation
    phase_bowlers = defaultdict(list)
    for bowler, phases in bowler_dict.items():
        for p in phases:
            phase_bowlers[p].append(bowler)

    # Helper function to check if a bowler can bowl a specific over
    def can_bowl(bowler, count, phase, pp_tracker, middle_tracker):
        if count[bowler] >= max_overs:
            return False
        if phase == 'P' and bowler in all_phase_bowlers and pp_tracker[bowler] >= 2:
            return False
        if phase == 'M' and bowler in all_phase_bowlers and middle_tracker[bowler] >= 1:
            return False
        return True

    # Start filling the bowling order
    sequence = [None] * 20  # Initialize a sequence with 20 slots
    count = Counter()  # Counter for the number of overs bowled by each bowler
    pp_tracker = Counter()  # To track how many overs a bowler has bowled in PP
    middle_tracker = Counter()  # To track how many overs a bowler has bowled in M
    
    # Identifying bowlers who are available for all 3 phases (P, M, D)
    all_phase_bowlers = [b for b, phases in bowler_dict.items() if 'P' in phases and 'M' in phases and 'D' in phases]
    
    # Prioritize bowlers for PowerPlay and pick top 2 for the first two overs
    top_p_bowlers = [b for b in bowler_dict if 'P' in bowler_dict[b]]
    top_p_bowlers.sort(key=lambda bowler: bowler_dict[bowler].count('P'), reverse=True)  # Prioritize based on P-phase count
    
    # Ensure the first two overs are bowled by the top two priority bowlers in PP
    sequence[0], sequence[1] = top_p_bowlers[0], top_p_bowlers[1]
    count[sequence[0]] += 1
    count[sequence[1]] += 1
    if sequence[0] in all_phase_bowlers:
        pp_tracker[sequence[0]] += 1
    if sequence[1] in all_phase_bowlers:
        pp_tracker[sequence[1]] += 1

    # Now fill the rest of the overs
    prev_bowler = sequence[1]  # Start from the third over
    for over in range(2, 20):
        phase = get_phase(over + 1)
        available_bowlers = [b for b in phase_bowlers[phase] if b != prev_bowler and can_bowl(b, count, phase, pp_tracker, middle_tracker)]
        if not available_bowlers:
            raise RuntimeError(f"No valid bowlers available for over {over + 1}")
        
        # Randomly shuffle the available bowlers and choose one
        chosen_bowler = random.choice(available_bowlers)
        sequence[over] = chosen_bowler
        count[chosen_bowler] += 1
        if phase == 'P' and chosen_bowler in all_phase_bowlers:
            pp_tracker[chosen_bowler] += 1
        if phase == 'M' and chosen_bowler in all_phase_bowlers:
            middle_tracker[chosen_bowler] += 1
        
        # Update the previous bowler
        prev_bowler = chosen_bowler
    
    return sequence


## new function, with deprioritization

In [291]:
def get_phase(over):
    if over <= 6:
        return 'P'
    elif over <= 15:
        return 'M'
    else:
        return 'D'
    
def generate_bowling_order(bowler_dict, max_overs=4):
    from collections import Counter, defaultdict
    import random

    # Phase-wise bowler allocation
    phase_bowlers = defaultdict(list)
    for bowler, phases in bowler_dict.items():
        for p in phases:
            phase_bowlers[p].append(bowler)

    # Assigning ranks based on bowler_dict order
    bowler_ranks = {bowler: rank for rank, bowler in enumerate(bowler_dict.keys(), start=1)}
    print(bowler_ranks)

    def can_bowl(bowler, count, phase, pp_tracker, middle_tracker):
        if count[bowler] >= max_overs:
            return False
        if phase == 'P' and bowler in all_phase_bowlers and pp_tracker[bowler] >= 2:
            return False
        if phase == 'M' and bowler in all_phase_bowlers and middle_tracker[bowler] >= 1:
            return False
        return True

    sequence = [None] * 20
    count = Counter()
    pp_tracker = Counter()
    middle_tracker = Counter()

    all_phase_bowlers = [b for b, phases in bowler_dict.items() if 'P' in phases and 'M' in phases and 'D' in phases]

    top_p_bowlers = [b for b in bowler_dict if 'P' in bowler_dict[b]]
    top_p_bowlers.sort(key=lambda bowler: bowler_dict[bowler].count('P'), reverse=True)

    sequence[0], sequence[1] = top_p_bowlers[0], top_p_bowlers[1]
    count[sequence[0]] += 1
    count[sequence[1]] += 1
    if sequence[0] in all_phase_bowlers:
        pp_tracker[sequence[0]] += 1
    if sequence[1] in all_phase_bowlers:
        pp_tracker[sequence[1]] += 1

    prev_bowler = sequence[1]
    for over in range(2, 20):
        phase = get_phase(over + 1)
        eligible = [b for b in phase_bowlers[phase] if b != prev_bowler and can_bowl(b, count, phase, pp_tracker, middle_tracker)]
        primary = [b for b in eligible if bowler_ranks[b] <= 6]
        backup = [b for b in eligible if bowler_ranks[b] > 6]
        available_bowlers = primary or backup

        if not available_bowlers:
            print(sequence)
            raise RuntimeError(f"No valid bowlers available for over {over + 1}")

        chosen_bowler = random.choice(available_bowlers)
        sequence[over] = chosen_bowler
        count[chosen_bowler] += 1
        if phase == 'P' and chosen_bowler in all_phase_bowlers:
            pp_tracker[chosen_bowler] += 1
        if phase == 'M' and chosen_bowler in all_phase_bowlers:
            middle_tracker[chosen_bowler] += 1

        prev_bowler = chosen_bowler

    return sequence

## applying

In [297]:
bowler_dict = {'Trent':['P','M'], 'Jass':['P','M','D'], 'Theek':['P','M','D'],'Mulani':['M'], 'Hard':['M','D'],
               #'Azmat':['P'], 
               #, '7':['M'], '8':['M']
               #, 'Venky':['M']
              }
# livi = death
# kuldeep = pp, middle
# bethell = middle
# porel = pp
# naveen = death
# badoni = middle
# venky = middle

attempt = False
while attempt == False:
    try:
        order = generate_bowling_order(bowler_dict)
        #print(order)
        attempt = True
    except:
        print("issue")
        continue
    

for i in range(len(order)):
    print(f"{i+1}th over --> {order[i]}")
    if i==5:
        print(" "*10)
        print("POWERPLAY over")
        print(" "*10)
    if i==14:
        print(" "*10)
        print("MIDDLE over")
        print(" "*10)
    if i==19:
        print(" "*10)
        print("INNING over")
        print(" "*10)

{'Trent': 1, 'Jass': 2, 'Theek': 3, 'Mulani': 4, 'Hard': 5}
['Trent', 'Jass', 'Theek', 'Trent', 'Theek', 'Jass', 'Theek', 'Jass', 'Hard', 'Trent', 'Hard', 'Trent', 'Mulani', 'Hard', 'Mulani', 'Jass', 'Hard', 'Theek', None, None]
issue
{'Trent': 1, 'Jass': 2, 'Theek': 3, 'Mulani': 4, 'Hard': 5}
['Trent', 'Jass', 'Theek', 'Jass', 'Theek', 'Trent', 'Theek', 'Mulani', 'Hard', 'Mulani', 'Trent', 'Hard', 'Trent', 'Mulani', 'Jass', 'Theek', 'Jass', 'Hard', None, None]
issue
{'Trent': 1, 'Jass': 2, 'Theek': 3, 'Mulani': 4, 'Hard': 5}
1th over --> Trent
2th over --> Jass
3th over --> Trent
4th over --> Theek
5th over --> Trent
6th over --> Jass
          
POWERPLAY over
          
7th over --> Mulani
8th over --> Jass
9th over --> Theek
10th over --> Mulani
11th over --> Trent
12th over --> Mulani
13th over --> Hard
14th over --> Mulani
15th over --> Hard
          
MIDDLE over
          
16th over --> Jass
17th over --> Hard
18th over --> Theek
19th over --> Hard
20th over --> Theek
          

In [298]:
pd.Series(order).value_counts()

Trent     4
Jass      4
Theek     4
Mulani    4
Hard      4
dtype: int64

In [296]:
order

['Hoff',
 'Max',
 'Hoff',
 'Max',
 'Azmat',
 'Max',
 'Swap',
 'Sandy',
 'Swap',
 'Dagar',
 'Sandy',
 'Dagar',
 'Swap',
 'Dagar',
 'Sandy',
 'Hoff',
 'Sandy',
 'Hoff',
 'Swap',
 'Dagar']

## applying

In [ ]:
team = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Teams/SQUADS_season_02.xlsx', 
                        sheet_name='SRH')
team = team.sort_values(by='XI').head(11)

bowling_team = team

bowlers_df = bowling_team.dropna(subset=['bowl'])[['bowl','Func_Name','phase']]\
                        .sort_values('bowl').reset_index(drop=True)
bowlers_df['bowl'] = bowlers_df['bowl'].astype(int)

#bowler_dict = dict(zip(bowlers_df['bowl'], bowlers_df['phase'].str.split(',')))
bowler_dict = dict(zip(bowlers_df['Func_Name'], bowlers_df['phase'].str.split(',')))

bowler_dict

In [ ]:
attempt = False
while attempt == False:
    try:
        order = generate_bowling_order(bowler_dict)
        #print(order)
        attempt = True
    except:
        continue
    

for i in range(len(order)):
    print(f"{i+1}th over --> {order[i]}")
    if i==5:
        print(" "*10)
        print("POWERPLAY over")
        print(" "*10)
    if i==14:
        print(" "*10)
        print("MIDDLE over")
        print(" "*10)
    if i==19:
        print(" "*10)
        print("INNING over")
        print(" "*10)

In [ ]:
def has_consecutive_duplicates(lst):
    return any(lst[i] == lst[i+1] for i in range(len(lst)-1))

has_consecutive_duplicates(order)